In [61]:
import os
import io
import pandas as pd
from pandas import DataFrame
import sys
import numpy as np
import twitter
from twitter import error
import json
import jsonpickle
import time
import random

import google.cloud
from google.cloud import datastore
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

from sqlalchemy import *
import sqlalchemy
import sqlalchemy.schema
from sqlalchemy.types import *

from IPython.display import *
from pandas.io.json import *
# gcloud auth application-default login

In [21]:
#client = datastore.Client()
#client = language.LanguageServiceClient()

In [80]:
api = twitter.Api(consumer_key="NOrVFOrKaGpOsD1LHjlDmD9lA",
                  consumer_secret="8Lg3hdsmIbEn9Q1fLTiWbCqHD7vp2OSc21mvTGElnudCr5jM7p",
                  access_token_key="49683260-3LQ6XoNAFi9R7MVaSGU1igwNxJkfxL6aFpUJfe7pG",
                  access_token_secret="zNXNyrewzVYC2kIqQ1GlGQoLIDHmTJtjXpMFZhwKGxYb8")

In [62]:
#https://dev.twitter.com/rest/reference/get/search/tweets
# https://python-twitter.readthedocs.io/en/latest/searching.html
# https://www.karambelkar.info/2015/01/how-to-use-twitters-search-rest-api-most-effectively.
results = api.GetSearch(raw_query="q=%23apple&result_type=mixed&since=2017-09-10&count=100&lang=en")
#for a in results:
#    if(a.coordinates is not None):
#print(a.coordinates)
    
forma = [(a.created_at, 
          a.text,
          a.retweet_count, 
          a.favorite_count,
          a.id, 
          a.user.name, 
          a.user.screen_name,
          a.user.description,
          a.user.id,
          a.user.location, 
          a.user.favourites_count, 
          a.user.followers_count,
          a.user.friends_count, 
          a.user.statuses_count, 
          a.user.created_at, 
          a.user.verified, 
          a.user.time_zone, 
          a.coordinates,
          a.in_reply_to_status_id, 
          a.in_reply_to_user_id, 
          a.in_reply_to_screen_name) if a.coordinates is None else (a.created_at, 
          a.text,
          a.retweet_count, 
          a.favorite_count,
          a.id, 
          a.user.name, 
          a.user.screen_name,
          a.user.description,
          a.user.id,
          a.user.location, 
          a.user.favourites_count, 
          a.user.followers_count,
          a.user.friends_count, 
          a.user.statuses_count, 
          a.user.created_at, 
          a.user.verified, 
          a.user.time_zone, 
          str(a.coordinates['coordinates']),
          a.in_reply_to_status_id, 
          a.in_reply_to_user_id, 
          a.in_reply_to_screen_name) for a in results]
#print(forma)
#https://dev.twitter.com/overview/api/tweets#obj-coordinates

In [65]:
sinceId = None
maxTweets = 10000000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits
max_id = -1
tweetCount = 0
tweet_list = []

In [66]:
while tweetCount <= 100:
    try:
        if(max_id <= 0):
            if (not sinceId):
                new_tweets = api.GetSearch(raw_query="q=%23apple&result_type=mixed&count=100&lang=en")
            else:
                new_tweets = api.GetSearch(raw_query="q=%23apple&result_type=mixed&count=100&lang=en",since_id=str(since_id),max_id=str(max_id-1))
        else:
            new_tweets = api.GetSearch(raw_query="q=%23apple&result_type=mixed&count=100&lang=en", since_id=str(since_id),max_id=str(max_id-1))       
        if not new_tweets:
            print("No more tweets found")
            break
        for tweet in new_tweets:
            #print(tweet)
            tweet_list.append(tweet)
        tweetCount += len(new_tweets)
        print("Downloaded {0} tweets".format(tweetCount))
        max_id = new_tweets[-1].id
        since_id = new_tweets[1].id
        print(max_id, "   ->  ", since_id)
    except api.error.TwitterError as e:
        # Just exit if any error
        print("some error : " + str(e))
        break

Downloaded 100 tweets
913721993748516864    ->   913442316207235072
Downloaded 200 tweets
913721993748516864    ->   913442316207235072
Downloaded 300 tweets
913721993748516864    ->   913442316207235072
Downloaded 400 tweets
913721993748516864    ->   913442316207235072
Downloaded 500 tweets
913722029039169536    ->   913442316207235072
Downloaded 600 tweets
913722029039169536    ->   913442316207235072
Downloaded 700 tweets
913722029039169536    ->   913442316207235072
Downloaded 800 tweets
913722029039169536    ->   913442316207235072
Downloaded 900 tweets
913722029039169536    ->   913442316207235072
Downloaded 1000 tweets
913722029039169536    ->   913442316207235072
Downloaded 1100 tweets
913722029039169536    ->   913442316207235072


In [75]:
''' Function that takes in a search string 'query', the maximum
        number of tweets 'max_tweets', and the minimum (i.e., starting)
        tweet id. It returns a list of tweepy.models.Status objects. '''
max_tweets = 500
max_id = 911741971454099457
since_id = 911379583580229632
searched_tweets = []

while len(searched_tweets) < max_tweets:
    remaining_tweets = max_tweets - len(searched_tweets)
    try:
        new_tweets = api.GetSearch(raw_query='q=%23apple&result_type=mixed&lang=en', count=remaining_tweets, since_id=str(since_id),max_id=str(max_id-1))
        print('found',len(new_tweets),'tweets')
        if not new_tweets:
            print('no tweets found')
            break
        searched_tweets.extend(new_tweets)
        max_id = new_tweets[-1].id
    except tweepy.TweepError:
        print('exception raised, waiting 15 minutes')
        break # stop the loop
#return searched_tweets, max_id

found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets
found 15 tweets


In [76]:
forma = [(a.created_at, 
          a.text,
          a.retweet_count, 
          a.favorite_count,
          a.id, 
          a.user.name, 
          a.user.screen_name,
          a.user.description,
          a.user.id,
          a.user.location, 
          a.user.favourites_count, 
          a.user.followers_count,
          a.user.friends_count, 
          a.user.statuses_count, 
          a.user.created_at, 
          a.user.verified, 
          a.user.time_zone, 
          a.coordinates,
          a.in_reply_to_status_id, 
          a.in_reply_to_user_id, 
          a.in_reply_to_screen_name) if a.coordinates is None else (a.created_at, 
          a.text,
          a.retweet_count, 
          a.favorite_count,
          a.id, 
          a.user.name, 
          a.user.screen_name,
          a.user.description,
          a.user.id,
          a.user.location, 
          a.user.favourites_count, 
          a.user.followers_count,
          a.user.friends_count, 
          a.user.statuses_count, 
          a.user.created_at, 
          a.user.verified, 
          a.user.time_zone, 
          str(a.coordinates['coordinates']),
          a.in_reply_to_status_id, 
          a.in_reply_to_user_id, 
          a.in_reply_to_screen_name) for a in searched_tweets]
print(forma)

[('Fri Sep 29 00:28:20 +0000 2017', 'Designed to protect privacy\n#Apple https://t.co/OA2qxWvGnj', 185, 619, 913561044932366336, 'Philip Schiller', 'pschiller', 'Apple, Illumina, Boston College, Sports, Cars, Science, Scuba, Drums, Photography. (fair warning: I block haters)', 17104751, 'One Infinite Loop ∞', 49, 230010, 168, 595, 'Sat Nov 01 19:58:49 +0000 2008', True, 'Pacific Time (US & Canada)', None, None, None, None), ('Thu Sep 28 16:36:33 +0000 2017', '.@FCC chairman urges #Apple to activate FM radios in iPhones in light of recent disasters https://t.co/Dlne9ECwxL https://t.co/mZQgKfVoSf', 17, 28, 913442316207235072, 'AppleInsider', 'appleinsider', 'In-depth Apple news, analysis & reviews since 1997. Facebook: https://t.co/4sonFyLhfu YouTube: https://t.co/R2aAX6Ml8d', 20542450, 'Cupertino, California', 1241, 435743, 37, 31492, 'Tue Feb 10 20:27:09 +0000 2009', True, 'Central Time (US & Canada)', None, None, None, None), ('Thu Sep 28 17:42:55 +0000 2017', "'#Apple community' pull

In [77]:
#https://github.com/agalea91/nhl_2016_playoffs_twitter/blob/master/nhl_playoffs_2016_twitter_study.ipynb
data = {'created_at': [],'text': [], 
        'retweet_count': [], 'tweet_id':[],
        'user_favorites_count': [], 'tweet_location': [],
        'user_followers_count':[], 'user_time_zone':[],
        'user_id':[], 'user_name':[], 'in_reply_to_status_id':[], 
        'in_reply_to_user_id':[], 'in_reply_to_screen_name':[],
        'screen_name':[], 'user_verified':[],
        'user_location':[], 'user_description':[], 
        'user_statuses_count': [], 'user_created_at':[],
        'user_friends_count': [], 'favorite_count': []}

for t in forma:
    data['created_at'].append(t[0])
    data['text'].append(t[1])
    data['retweet_count'].append(t[2])
    data['favorite_count'].append(t[3])
    data['tweet_id'].append(t[4])
    data['user_name'].append(t[5])
    data['screen_name'].append(t[6])
    data['user_description'].append(t[7])
    data['user_id'].append(t[8])
    data['user_location'].append(t[9])
    data['user_favorites_count'].append(t[10])
    data['user_followers_count'].append(t[11])
    data['user_friends_count'].append(t[12])
    data['user_statuses_count'].append(t[13])
    data['user_created_at'].append(t[14])
    data['user_verified'].append(t[15])
    data['user_time_zone'].append(t[16])
    data['tweet_location'].append(t[17])
    data['in_reply_to_status_id'].append(t[18])
    data['in_reply_to_user_id'].append(t[19])
    data['in_reply_to_screen_name'].append(t[20])

df = pd.DataFrame(data)
#df.drop_duplicates(inplace=True)
df['created_at'] = pd.to_datetime(df['created_at'])
df['user_created_at'] = pd.to_datetime(df['user_created_at'])

RT = []
for t in data['text']:
    RT.append(t.split()[0]=='RT')
df['RT'] = RT

df.head()

,created_at,favorite_count,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweet_count,screen_name,text,tweet_id,tweet_location,...,user_favorites_count,user_followers_count,user_friends_count,user_id,user_location,user_name,user_statuses_count,user_time_zone,user_verified,RT
0,2017-09-29 00:28:20,619,None,None,None,185,pschiller,Designed to protect privacy\n#Apple https://t....,913561044932366336,None,...,49,230010,168,17104751,One Infinite Loop ∞,Philip Schiller,595,Pacific Time (US & Canada),True,False
1,2017-09-28 16:36:33,28,None,None,None,17,appleinsider,.@FCC chairman urges #Apple to activate FM rad...,913442316207235072,None,...,1241,435743,37,20542450,"Cupertino, California",AppleInsider,31492,Central Time (US & Canada),True,False
2,2017-09-28 17:42:55,22,None,None,None,19,appleinsider,'#Apple community' pulls in $13M for disaster ...,913459018135465985,None,...,1241,435743,37,20542450,"Cupertino, California",AppleInsider,31492,Central Time (US & Canada),True,False
3,2017-09-29 11:35:00,0,None,None,None,0,mathewderric,We don't want a fake control centre make it wo...,913728813770489856,None,...,94,573,345,68389164,"Kerala, India",Mathew Derric,170,Chennai,False,False
4,2017-09-29 11:34:30,0,None,None,None,0,ThatLDNFella,Apple have gone too far...the new iOS is vile....,913728688050573312,None,...,203,255,611,895741051142422528,"London, England",Firmin.,485,None,False,False


In [71]:
df.describe()

,favorite_count,in_reply_to_status_id,in_reply_to_user_id,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count
count,1000.00000,2.000000e+01,2.000000e+01,1000.000000,1.000000e+03,1000.000000,1000.000000,1000.000000,1.000000e+03,1.000000e+03
mean,6.90100,9.125628e+17,4.738930e+07,20.172000,9.137184e+17,6505.643000,15084.851000,2368.991000,1.756898e+17,5.942340e+04
std,61.65597,1.188100e+15,3.374335e+07,87.326401,4.167162e+13,17679.093754,64864.437669,6996.131109,3.324609e+17,1.768640e+05
min,0.00000,9.114048e+17,1.450035e+07,0.000000,9.134423e+17,0.000000,0.000000,0.000000,6.961630e+05,4.000000e+00
25%,0.00000,9.114048e+17,1.450035e+07,0.000000,9.137240e+17,41.000000,177.000000,167.750000,1.108057e+08,1.373000e+03
50%,0.00000,9.125628e+17,4.738930e+07,1.000000,9.137256e+17,467.500000,637.500000,404.000000,4.688636e+08,6.967000e+03
75%,0.00000,9.137208e+17,8.027824e+07,8.000000,9.137269e+17,4241.000000,2818.000000,1898.000000,3.725919e+09,2.870375e+04
max,619.00000,9.137208e+17,8.027824e+07,792.000000,9.137280e+17,142196.000000,435736.000000,61869.000000,9.136429e+17,1.180371e+06


In [78]:
len(np.unique(df.tweet_id))

15

In [73]:
df.columns[df.isnull().any()]

Index(['in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_user_id', 'tweet_location', 'user_time_zone'],
      dtype='object')

In [31]:
#https://cloud.google.com/natural-language/docs/sentiment-tutorial

In [79]:
engine = create_engine('mysql+pymysql://root:@localhost/sawi_tweets?charset=utf8mb4', encoding='utf8', echo = True)
df.to_sql('sawi_tweets_historical', engine, if_exists='replace')

2017-09-29 13:36:00,629 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2017-09-29 13:36:00,630 INFO sqlalchemy.engine.base.Engine {}
2017-09-29 13:36:00,641 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2017-09-29 13:36:00,647 INFO sqlalchemy.engine.base.Engine {}
2017-09-29 13:36:00,654 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8' and `Collation` = 'utf8_bin'
2017-09-29 13:36:00,658 INFO sqlalchemy.engine.base.Engine {}
2017-09-29 13:36:00,666 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2017-09-29 13:36:00,682 INFO sqlalchemy.engine.base.Engine {}
2017-09-29 13:36:00,691 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2017-09-29 13:36:00,696 INFO sqlalchemy.engine.base.Engine {}
2017-09-29 13:36:00,702 INFO sqlalchemy.engine.base.Engine SELECT CAST('test collated returns' AS CHAR CHARACTER SET utf8) COLLATE utf8_bin AS anon_1
2017-09-29 13

2017-09-29 13:36:01,137 INFO sqlalchemy.engine.base.Engine COMMIT


In [46]:
import json
from pprint import pprint

#with open('/home/jm/Documents/te/twitter_search/#apple/#apple_2017-09-23.json') as data_file:    
  #  data = json.load(data_file)

#pprint(data)
tweets = []
for line in open('/home/jm/Documents/te/twitter_search/#apple/#apple_2017-09-23.json', 'r'):
    tweets.append(json.loads(line))

In [49]:
df = pd.DataFrame()
 
df['text'] = list(map(lambda tweet: tweet['text'], tweets))

df['location'] = list(map(lambda tweet: tweet['user']['location'], tweets))

df['country_code'] = list(map(lambda tweet: tweet['place']['country_code']
                              if tweet['place'] != None else '', tweets))

df['long'] = list(map(lambda tweet: tweet['coordinates']['coordinates'][0]
                    if tweet['coordinates'] != None else 'NaN', tweets))

df['latt'] = list(map(lambda tweet: tweet['coordinates']['coordinates'][1]
                    if tweet['coordinates'] != None else 'NaN', tweets))
df.describe()

,text,location,country_code,long,latt
count,4963,4963,4963,4963,4963
unique,3647,1923,35,60,60
top,RT @ChakerKhazaal: #Palestine finally arrives ...,,,NaN,NaN
freq,247,1252,4785,4900,4900


In [44]:
formasss = [(a.created_at, 
          a.text,
          a.retweet_count, 
          a.favorite_count,
          a.id, 
          a.user.name, 
          a.user.screen_name,
          a.user.description,
          a.user.id,
          a.user.location, 
          a.user.favourites_count, 
          a.user.followers_count,
          a.user.friends_count, 
          a.user.statuses_count, 
          a.user.created_at, 
          a.user.verified, 
          a.user.time_zone, 
          a.in_reply_to_status_id, 
          a.in_reply_to_user_id, 
          a.in_reply_to_screen_name) for a in tweets]

AttributeError: 'dict' object has no attribute 'created_at'

In [ ]:
data = {'created_at': [],'text': [], 
        'retweet_count': [], 'tweet_id':[],
        'user_favorites_count': [], 'tweet_location': [],
        'user_followers_count':[], 'user_time_zone':[],
        'user_id':[], 'user_name':[], 'in_reply_to_status_id':[], 
        'in_reply_to_user_id':[], 'in_reply_to_screen_name':[],
        'screen_name':[], 'user_verified':[],
        'user_location':[], 'user_description':[], 
        'user_statuses_count': [], 'user_created_at':[],
        'user_friends_count': [], 'favorite_count': []}

for t in forma:
    data['created_at'].append(t[0])
    data['text'].append(t[1])
    data['retweet_count'].append(t[2])
    data['favorite_count'].append(t[3])
    data['tweet_id'].append(t[4])
    data['user_name'].append(t[5])
    data['screen_name'].append(t[6])
    data['user_description'].append(t[7])
    data['user_id'].append(t[8])
    data['user_location'].append(t[9])
    data['user_favorites_count'].append(t[10])
    data['user_followers_count'].append(t[11])
    data['user_friends_count'].append(t[12])
    data['user_statuses_count'].append(t[13])
    data['user_created_at'].append(t[14])
    data['user_verified'].append(t[15])
    data['user_time_zone'].append(t[16])
    data['tweet_location'].append(t[17])
    data['in_reply_to_status_id'].append(t[18])
    data['in_reply_to_user_id'].append(t[19])
    data['in_reply_to_screen_name'].append(t[20])
    

In [42]:
print(tweets)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

